In [2]:
import pandas as pd
import numpy as np

In [3]:
import json
from pathlib import Path

def load_json(file_path, encoding='utf-8', verbose=True, extract_key=None):
    try:
        file_path = Path(file_path)
        
        if verbose:
            print(f"📂 Loading: {file_path}")
        
        if not file_path.exists():
            return None
        
        with open(file_path, 'r', encoding=encoding) as f:
            data = json.load(f)
        if extract_key:
            if isinstance(data, dict) and extract_key in data:
                if verbose:
                    print(f"🔑 Extracting key: '{extract_key}'")
                data = data[extract_key]
            else:
                print(f"⚠️ Key '{extract_key}' not found in JSON")
        if verbose:
            data_type = type(data).__name__
            
            if isinstance(data, list):
                print(f"✅ Loaded {len(data)} items (list)")
            elif isinstance(data, dict):
                print(f"✅ Loaded dictionary with keys: {list(data.keys())}")
                # Nếu dict có nested list, show thêm thông tin
                for key, value in data.items():
                    if isinstance(value, list):
                        print(f"   - {key}: {len(value)} items")
            else:
                print(f"✅ Loaded {data_type}")
        
        return data
        
    except json.JSONDecodeError as e:
        return None
    except Exception as e:
        return None


In [4]:
def load_historical_queries(json_path):

    print(f"📂 Loading from: {json_path}")
    
    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    print(f"📊 Raw data type: {type(data)}")
    
    # Case 1: Already a list
    if isinstance(data, list):
        queries_list = data
    
    # Case 2: Dict with nested structure
    elif isinstance(data, dict):
        # Try common keys
        for key in ['queries', 'query', 'data', 'results']:
            if key in data:
                queries_list = data[key]
                if not isinstance(queries_list, list):
                    queries_list = [queries_list]
                break
        else:
            # No known key, treat whole dict as single query
            queries_list = [data]
    
    else:
        raise ValueError(f"Unsupported data type: {type(data)}")
    
    print(f"✅ Loaded {len(queries_list)} queries")
    
    # Convert to DataFrame
    df = pd.DataFrame(queries_list)
    
    print(f"📊 DataFrame shape: {df.shape}")
    print(f"📋 Columns: {list(df.columns)}")
    
    return df


In [36]:
df = load_json('../data/processed/data.json')
df = pd.DataFrame(df)
df = df.drop(columns='subAddress')
print(df)

📂 Loading: ..\data\processed\data.json
✅ Loaded 1018 items (list)
     venueId            phuong       district                   city  \
0      v-001   Phường Tân Hưng         Quận 7  Thành phố Hồ Chí Minh   
1      v-001   Phường Tân Hưng         Quận 7  Thành phố Hồ Chí Minh   
2      v-001   Phường Tân Hưng         Quận 7  Thành phố Hồ Chí Minh   
3      v-002     Xã Phước Kiển   Huyện Nhà Bè  Thành phố Hồ Chí Minh   
4      v-002     Xã Phước Kiển   Huyện Nhà Bè  Thành phố Hồ Chí Minh   
...      ...               ...            ...                    ...   
1013   v-248  Phường Dịch Vọng  Quận Cầu Giấy       Thành phố Hà Nội   
1014   v-248  Phường Dịch Vọng  Quận Cầu Giấy       Thành phố Hà Nội   
1015   v-248  Phường Dịch Vọng  Quận Cầu Giấy       Thành phố Hà Nội   
1016   v-248  Phường Dịch Vọng  Quận Cầu Giấy       Thành phố Hà Nội   
1017   v-248  Phường Dịch Vọng  Quận Cầu Giấy       Thành phố Hà Nội   

                                categoryId     category start_time en

In [38]:
import sys
import os

from services.data_services.extract_category import ExtractCategory
sys.path.append(os.path.dirname(os.getcwd()))
from services.data_services.detect_district import DetectDistrict
from services.query_service.extract_category import ExtractSport
import sys
import os
import importlib

# Lấy thư mục gốc project (thư mục chứa "services")
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(PROJECT_ROOT)

# Reload module để chắc chắn lấy file mới
import services.query_service.extract_category as extractSport
import services.query_service.extract_location as extract_location
import services.query_service.extract_time as extract_time
importlib.reload(extract_location)
importlib.reload(extractSport)
importlib.reload(extract_time)
from services.query_service.extract_location import ExtractLocation
from services.query_service.extract_category import ExtractSport
from services.query_service.extract_time import ExtractTime

df['phuong'] = df.apply(DetectDistrict.get_ward_info, axis=1)
df['district'] = df.apply(DetectDistrict.get_district_info, axis=1)
df['city'] = df.apply(DetectDistrict.get_city_info, axis=1)
df = df.drop(columns='category')
print (df)

     venueId      phuong  district         city  \
0      v-001    Tân Hưng         7  Hồ Chí Minh   
1      v-001    Tân Hưng         7  Hồ Chí Minh   
2      v-001    Tân Hưng         7  Hồ Chí Minh   
3      v-002  Phước Kiển    Nhà Bè  Hồ Chí Minh   
4      v-002  Phước Kiển    Nhà Bè  Hồ Chí Minh   
...      ...         ...       ...          ...   
1013   v-248   Dịch Vọng  Cầu Giấy       Hà Nội   
1014   v-248   Dịch Vọng  Cầu Giấy       Hà Nội   
1015   v-248   Dịch Vọng  Cầu Giấy       Hà Nội   
1016   v-248   Dịch Vọng  Cầu Giấy       Hà Nội   
1017   v-248   Dịch Vọng  Cầu Giấy       Hà Nội   

                                categoryId start_time end_time  
0     a1f3b6e4-2c9a-4c1b-b8a2-9c0d4fe71201      05:00    22:00  
1     a1f3b6e4-2c9a-4c1b-b8a2-9c0d4fe71201      05:00    22:00  
2     a1f3b6e4-2c9a-4c1b-b8a2-9c0d4fe71201      05:00    22:00  
3     a1f3b6e4-2c9a-4c1b-b8a2-9c0d4fe71201      05:00    22:00  
4     a1f3b6e4-2c9a-4c1b-b8a2-9c0d4fe71201      05:00    22:00

In [41]:
import sys
import importlib

# Xóa module khỏi cache
sys.modules.pop('services.parse_user_query', None)

In [42]:
# Reload module
import services.parse_user_query as parse_module
importlib.reload(parse_module)

from services.parse_user_query import ParseUtils

In [43]:
# Cell 1: Import libraries
import pandas as pd
import numpy as np

def create_training_data(venues_df, historical_queries):
    training_data = []
    if not isinstance(historical_queries, pd.DataFrame):
        historical_queries = pd.DataFrame(historical_queries)


    for idx, query_record in historical_queries.iterrows():
        query = query_record["query"]
        clicked_venue_id = query_record.get("clicked_venue_id") or query_record.get("venueId")
        for _, venue_row in venues_df.iterrows():
            
            features = ParseUtils.pair_feature(parsed_query=query_record, venue_row=venue_row)
            # default_features = ["", "", "", "", ""]
            # features = features + default_features[len(features):]
            venue_id = venue_row.get("venueId") or venue_row.get("venueid") or venue_row.get("id")
            label = 1 if str(venue_id) == str(clicked_venue_id) else 0
            record = {
                "sport_match": features['sport_match'],
                "location_match" : features['location_match'],
                "time_match": features['time_match'],
                "label": label,
                "query": query,
                "venue_id": venue_id,
            }

            training_data.append(record)

    df_training = pd.DataFrame(training_data)
    return df_training

In [33]:
import json
import pandas as pd

with open('../data/raw/query.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

if isinstance(data, dict) and 'query' in data:
    historical_queries = pd.DataFrame(data['query'])
elif isinstance(data, list):
    historical_queries = pd.DataFrame(data)
else:
    historical_queries = pd.json_normalize(data)

print(historical_queries)

                                        query  \
0       sân Pickleball ở Quận 7 vào buổi sáng   
1        sân Cầu lông Huyện Nhà Bè buổi chiều   
2           sân Tennis ở Quận 8 vào buổi sáng   
3    tìm sân Bóng Rổ Quận Tân Bình buổi chiều   
4       sân Pickleball Huyện Hóc Môn sáng nay   
5      sân Bóng chuyền Quận Gò Vấp buổi chiều   
6     sân Cầu lông Quận Tân Phú vào buổi sáng   
7   tìm sân Pickleball Quận 12 vào buổi chiều   
8                     sân Tennis Quận 11 sáng   
9        sân Pickleball Quận 1 vào buổi chiều   
10             sân Cầu lông Đống Đa buổi sáng   
11             sân Bóng Rổ Hà Đông buổi chiều   
12                sân Bóng chuyền Tây Hồ sáng   
13    sân Pickleball Quận Hoàng Mai buổi sáng   
14             sân Tennis Long Biên chiều nay   
15             sân Cầu lông Quận 9 buổi chiều   
16        tìm sân Bóng chuyền Quận 8 sáng sớm   
17           sân Pickleball Quận 5 buổi chiều   
18             sân Cầu lông Quận Tân Phú sáng   
19               sân

In [40]:
import sys
import os
import importlib

# Lấy thư mục gốc project (thư mục chứa "services")
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(PROJECT_ROOT)

# Reload module để chắc chắn lấy file mới
import services.query_service.extract_category as extractSport
import services.query_service.extract_location as extract_location
import services.query_service.extract_time as extract_time
importlib.reload(extract_location)
importlib.reload(extractSport)
importlib.reload(extract_time)
from services.query_service.extract_location import ExtractLocation
from services.query_service.extract_category import ExtractSport
from services.query_service.extract_time import ExtractTime



historical_queries = ExtractLocation.merge_query_with_locations(historical_queries)

historical_queries = ExtractSport.extract_category_from_df(historical_queries)
historical_queries = ExtractTime.extract_time_from_df(historical_queries)
historical_queries.head()




,query,viewed_venues,clicked_venue_id,booked,phuong,district,city,category,time
0,sân Pickleball ở Quận 7 vào buổi sáng,"[v-001, v-081, v-029, v-217]",v-001,True,,7,Hồ Chí Minh,e3a7b9d9-5c2f-4d33-b4c4-33f1d765cd02,morning
1,sân Cầu lông Huyện Nhà Bè buổi chiều,"[v-002, v-081, v-014]",v-002,False,,Nhà Bè,Hồ Chí Minh,a1f3b6e4-2c9a-4c1b-b8a2-9c0d4fe71201,afternoon
2,sân Tennis ở Quận 8 vào buổi sáng,"[v-003, v-024, v-079, v-080]",v-003,True,,8,Hồ Chí Minh,e3a7b9d9-5c2f-4d33-b4c4-33f1d765cd04,morning
3,tìm sân Bóng Rổ Quận Tân Bình buổi chiều,"[v-004, v-008, v-009, v-055]",v-004,True,,Tân Bình,Hồ Chí Minh,e3a7b9d9-5c2f-4d33-b4c4-33f1d765cd03,afternoon
4,sân Pickleball Huyện Hóc Môn sáng nay,"[v-005, v-006, v-007]",v-005,False,,Hóc Môn,Hồ Chí Minh,e3a7b9d9-5c2f-4d33-b4c4-33f1d765cd02,morning


In [44]:
df_training = create_training_data (venues_df=df, historical_queries= historical_queries)


In [45]:
print (df_training)

       sport_match  location_match  time_match  label  \
0                0               1           1      1   
1                0               1           1      1   
2                0               1           1      1   
3                0               1           1      0   
4                0               1           1      0   
...            ...             ...         ...    ...   
60057            0               0           1      0   
60058            0               0           1      0   
60059            0               0           1      0   
60060            0               0           1      0   
60061            0               0           1      0   

                                       query venue_id  
0      sân Pickleball ở Quận 7 vào buổi sáng    v-001  
1      sân Pickleball ở Quận 7 vào buổi sáng    v-001  
2      sân Pickleball ở Quận 7 vào buổi sáng    v-001  
3      sân Pickleball ở Quận 7 vào buổi sáng    v-002  
4      sân Pickleball ở Quận 7 vào 

In [46]:
df_training.to_csv('../datasets/ground.csv')
historical_queries.to_csv('../datasets/historical_queries.csv')
df.to_csv('../datasets/df.csv')